# Data Preprocessing

This notebook builds a master dataframe of financial journals, clean the data, and export data to csv files.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
from nltk.stem.snowball import SnowballStemmer
from sklearn.preprocessing import StandardScaler
import regex as re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Load Dataset

In [2]:
# get a list of csv file in folder Dataset
dataset_p = '/content/drive/MyDrive/Trending-Topics-Dashboard-main/Dataset/Journal_of_Financial_Economics_{}.csv'

# read each csv file into a Pandas dataframe
# and store the dataframes into a list
list_df = [pd.read_csv(dataset_p.format(y), encoding='ISO-8859-1') for y in range(2020,1973,-1)]

# display number of dataframes created
print("Number of Data Frames:", len(list_df))

# look at the first dataframe
list_df[0].head()

FileNotFoundError: ignored

In [ ]:
# merge all dataframes into a master dataframe
df_master = pd.concat(list_df, axis=0, ignore_index=True)

# drop rows that have missing values
df_master.dropna(axis=0, how='any',inplace=True)

# get the name of first author
df_master['author_first'] = df_master.authors.apply(lambda x: x[1:-1].split(',')[0][1:-1])

# duplicate Year
df_master['year_master'] = df_master.year.copy()

# compute the standard deviation for Year
df_master['year_std'] = df_master.year / df_master.year.std()

# apply standard scaler on Year
df_master['year'] = StandardScaler().fit_transform(df_master['year'].values.reshape(-1, 1))

# display the shape of df_masters
print("Data's shape: ", df_master.shape)

# look at df_master
df_master.head()

Data's shape:  (2973, 14)


,title,abstract,authors,keywords,affiliations,month,link,journal_name,year,volume,issue,author_first,year_master,year_std
0,Empirical analysis of corporate tax reforms: W...,"Absent theoretical guidance, empiricists have ...","['Christopher A. Hennessy', 'Akitada Kasahara'...","['Capital structure', 'Corporate taxation', 'D...",[],March,https://www.sciencedirect.com//science/article...,Journal of Financial Economics,1.243352,135,3,Christopher A. Hennessy,2020,169.971142
1,Leveraged buyouts and bond credit spreads,Recent decades have witnessed several waves of...,"['Yael Eisenthal-Berkovitz', 'Peter FeldhÃ¼tte...","['Credit spreads', 'LBO risk', 'Structural mod...",[],March,https://www.sciencedirect.com//science/article...,Journal of Financial Economics,1.243352,135,3,Yael Eisenthal-Berkovitz,2020,169.971142
2,Do fire sales create externalities?,We develop three novel measures of the incenti...,"['Sergey Chernenko', 'Adi Sunderam']","['Fire sales', 'Liquidity management', 'Mutual...",[],March,https://www.sciencedirect.com//science/article...,Journal of Financial Economics,1.243352,135,3,Sergey Chernenko,2020,169.971142
3,Betting against correlation: Testing theories ...,We test whether the low-risk effect is driven ...,"['Cliff Asness', 'Andrea Frazzini', 'Niels Joa...","['Asset pricing', 'Leverage constraints', 'Lot...",[],March,https://www.sciencedirect.com//science/article...,Journal of Financial Economics,1.243352,135,3,Cliff Asness,2020,169.971142
4,Are early stage investors biased against women?,We study whether early stage investors have ge...,"['Michael Ewens', 'Richard R. Townsend']","['Gender gap', 'Entrepreneurship', 'Angel inve...",[],March,https://www.sciencedirect.com//science/article...,Journal of Financial Economics,1.243352,135,3,Michael Ewens,2020,169.971142


## Data Preprocessing & Cleaning

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

stops = set(stopwords.words("english"))    # get unique English stopwords
regex = re.compile(r'[^\w\s]+')            # regular expression pattern
stemmer = SnowballStemmer("english")       # create a Snowball Stemmer object for stemming

# duplicate a subset of the master dataframe
df = df_master[['year_master', 'year', 'year_std', 'month', 'keywords', 'abstract']]
df.rename(columns={'year_master': 'Year', 'year': 'Year_Scaled', 
                   'year_std': 'Year_STD', 'month': 'Month', 'keywords': 'Keywords',
                   'abstract': 'Abstract'}, inplace=True)

# clean the abstract
df['Abstract_Cleaned'] = df.Abstract.map(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))
df['Abstract_Cleaned'] = df.Abstract_Cleaned.map(lambda x: ' '.join([y for y in x.split(' ') if not y in stops]))
df['Abstract_Cleaned'] = df.Abstract_Cleaned.map(lambda x: ' '.join([regex.sub('', x) for x in x.split(' ')]))

# get the length of the abstract
df['Abstract Length'] = df.Abstract.str.len()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def stem_keywords(keywords):
    # create an empty list of keywords
    kw_list = []
    
    # iterate through each keyword in the list and apply stemming
    for kw in keywords[1:-1].split(', '):
        # stem each word in the  keyword
        stemmed_kw = ' '.join(stemmer.stem(word) for word in kw.split(' '))
        kw_list.append(stemmed_kw)
    
    return kw_list


# apply stemming to Keywords
df['Keywords_Cleaned'] = df['Keywords'].apply(stem_keywords)

In [ ]:
# count number of keywords
df['Number of Keywords'] = df['Keywords_Cleaned'].map(lambda keywords: \
                                                      0 if str(keywords) == "['']" else len(keywords))

### Month

In [ ]:
df['Month'] = df['Month'].replace('â\x80\x93', '-', regex=True)

In [ ]:
# display unique value of month
df['Month'].unique()

array(['March', 'February', 'January', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December',
       'November-December', '2000', 'October-November', '1992', '1989',
       'January-February', 'June-September', 'January-March'],
      dtype=object)

In [ ]:
import calendar

def process_month(month):
    '''
    - Convert month name to numeric value
    - Set any invalid month value to -1
    '''
    
    # extract month from a string
    month_extracted = month.split('-')[-1]
    
    # assume month's value is invalid
    month_numeric = -1
    
    if month_extracted.isnumeric() and int(month_extracted) in np.arange(1, 13):
        # month_extracted has value between 1 and 12
        month_numeric = int(month_extracted)
    elif not month_extracted.isnumeric():
        # month_extracted have text value "January" to "December"
        month_numeric = list(calendar.month_name).index(month_extracted)
        
    return month_numeric

In [ ]:
# clean month value
df['Month_Cleaned'] = df['Month'].map(process_month)

# take a look at data
df.head()

,Year,Year_Scaled,Year_STD,Month,Keywords,Abstract,Abstract_Cleaned,Abstract Length,Keywords_Cleaned,Number of Keywords,Month_Cleaned
0,2020,1.243352,169.971142,March,"['Capital structure', 'Corporate taxation', 'D...","Absent theoretical guidance, empiricists have ...",absent theoret guidance empiricist forc reli u...,1051,"[capit structur, corpor taxat, difference-in-d...",5,3
1,2020,1.243352,169.971142,March,"['Credit spreads', 'LBO risk', 'Structural mod...",Recent decades have witnessed several waves of...,recent decad wit sever wave buyout activity fi...,584,"[credit spread, lbo risk, structur model, leve...",4,3
2,2020,1.243352,169.971142,March,"['Fire sales', 'Liquidity management', 'Mutual...",We develop three novel measures of the incenti...,develop three novel measur incent equiti mutua...,586,"[fire sale, liquid manag, mutual fund]",3,3
3,2020,1.243352,169.971142,March,"['Asset pricing', 'Leverage constraints', 'Lot...",We test whether the low-risk effect is driven ...,test whether lowrisk effect driven leverag con...,861,"[asset price, leverag constraint, lotteri dema...",5,3
4,2020,1.243352,169.971142,March,"['Gender gap', 'Entrepreneurship', 'Angel inve...",We study whether early stage investors have ge...,studi whether earli stage investor gender bias...,744,"[gender gap, entrepreneurship, angel investor,...",4,3


## Export Data

In [ ]:
df_master.to_csv('/content/drive/MyDrive/Trending-Topics-Dashboard-main/data/data_master.csv', index=False)               # save master data
df.to_csv('/content/drive/MyDrive/Trending-Topics-Dashboard-main/data/data_cleaned.csv', index=False)                     # save cleaned data